# Image Segmentation
This section is responsible for detecting the various components of the image such as the lumbar and Spinal Canal

## Processing of Manual Annotation data
Converting the annotation files from `CVAT.xml` to YoloV8 Compatible format

In [ ]:
import os
import xml.etree.ElementTree as ET

In [ ]:
def convert_cvat_polyline_to_yolo(xml_path, output_dir):
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Get all unique class names from the polylines
    class_names = sorted(set(
        poly.get("label")
        for image in root.findall("image")
        for poly in image.findall("polygon")
    ))
    class_name_to_id = {name: idx for idx, name in enumerate(class_names)}
    print("Class mapping:", class_name_to_id)

    for image in root.findall("image"):
        img_name = image.get("name")
        img_width = int(image.get("width"))
        img_height = int(image.get("height"))

        label_lines = []
        for poly in image.findall("polygon"):
            label = poly.get("label")
            class_id = class_name_to_id[label]
            point_list = poly.get("points").split(";")

            points = [tuple(map(float, p.split(","))) for p in point_list]
            xs, ys = zip(*points)

            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)

            x_center = (x_min + x_max) / 2 / img_width
            y_center = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Write YOLO annotation to a text file
        txt_filename = os.path.splitext(img_name)[0] + ".txt"
        with open(os.path.join(output_dir, txt_filename), "w") as f:
            f.write("\n".join(label_lines))
    print(f"files saved successfully")

# Example usage:
xml_input_path = "annotations/annotations_b2.xml"         # Replace with your XML file path
yolo_output_folder = "yolo_labels_b2"         # Replace with your desired output folder
convert_cvat_polyline_to_yolo(xml_input_path, yolo_output_folder)


## Training the YOLO model to recognize the segments in the image

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.yaml")

In [ ]:
results = model.train(data="config.yaml", epochs=50)

In [ ]:
model.names

In [ ]:
results2 = model.predict(source='data/images/train', save=True)

In [ ]:
newmodel = YOLO("runs/detect/train6/weights/best.pt")
#result2 = newmodel.predict(source="path/to/image.jpg", conf=0.05, save=True)

In [ ]:
result2 = newmodel.predict(source="OLD",  imgsz=640)
# print(results[0].boxes.cls)   # See what classes are detected
# print(results[0].boxes.conf)  # See confidence scores

In [ ]:
results2

In [ ]:
import os
import csv

# Set the folder path
new_paths = ["dataset/b1/images/", "dataset/b2/images/", "dataset/b5/images/"]
# Set the output CSV path
output_csv = 'file_names.csv'
with open(output_csv, mode='w', newline='') as file:
    file_names =[]
    for folder_path in new_paths:
        #folder_path = each  # <-- change this to your folder path
        
        # Get list of all files (excluding directories)
        print(len(os.listdir(folder_path)), folder_path)
        file_a = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        print(len(file_a))
        file_names += file_a
        
    writer = csv.writer(file)
    writer.writerow(['File Name'])  # header
    for name in file_names:
        writer.writerow([name])

print(f"✅ {len(file_names)} file names saved to '{output_csv}'")


In [69]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
# Load data
df = pd.read_csv("file_names.csv")
fault_columns = ['A', 'B', 'C', 'D', 'E']
df[fault_columns] = df[fault_columns].replace({'T': 1, 'F': 0})
df['Image'] = df['Image'].apply(lambda x: f"data/images/train/{x}")

In [28]:
df.head()

,Image,A,B,C,D,E
0,data/images/train/AH000438.JPG,1,0,1,1,0
1,data/images/train/AH015875.JPG,0,0,0,1,0
2,data/images/train/AH019263.JPG,1,0,1,1,0
3,data/images/train/AH020596.JPG,1,0,0,1,0
4,data/images/train/AH026019.JPG,0,0,0,0,0


In [36]:
# Data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.1)

def get_data_generators(df, fault, datagen, target_size=(299, 299), batch_size=16):
    """
    Returns training and validation generators for a specific fault column.
    
    Parameters:
    - df: pandas DataFrame containing image paths and fault labels
    - fault: the fault column name (e.g., 'A')
    - datagen: an instance of ImageDataGenerator with preprocessing and split
    - target_size: desired image size (default: (299, 299))
    - batch_size: number of images per batch (default: 16)
    
    Returns:
    - train_gen: training data generator
    - val_gen: validation data generator
    """
    train_gen = datagen.flow_from_dataframe(
        dataframe=df,
        x_col='Image',
        y_col=fault,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        subset='training',
        shuffle=False
    )

    val_gen = datagen.flow_from_dataframe(
        dataframe=df,
        x_col='Image',
        y_col=fault,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        subset='validation',
        shuffle=False
    )

    return train_gen, val_gen

In [70]:
results = {}

for fault in fault_columns:
    print(f"\n🔍 Training for Fault: {fault}")

    # Use entire data as both train and val (no subset)
    train_gen, val_gen = get_data_generators(df, fault, datagen)
    val_gen = train_gen  # Using train_gen for validation

    # Build model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Train
    history = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=1)

    # Save model
    model.save(f"models/model_{fault}.keras")

    # Save accuracy and loss plots
    os.makedirs("metrics", exist_ok=True)

    plt.figure()
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{fault} Accuracy')
    plt.legend()
    plt.savefig(f"metrics/{fault}_accuracy.png")
    plt.close()

    plt.figure()
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{fault} Loss')
    plt.legend()
    plt.savefig(f"metrics/{fault}_loss.png")
    plt.close()

    # Predict and evaluate on same data
    val_gen.reset()
    preds = model.predict(val_gen).ravel()
    preds_binary = (preds > 0.5).astype(int)
    true_vals = val_gen.labels

    loss, acc = model.evaluate(val_gen, verbose=0)
    cm = confusion_matrix(true_vals, preds_binary)
    report = classification_report(true_vals, preds_binary, output_dict=True)
    report_text = classification_report(true_vals, preds_binary)

    results[fault] = {"loss": loss, "accuracy": acc, "classification_report": report}

    # Save classification report
    with open(f"metrics/{fault}_report.txt", "w") as f:
        f.write(report_text)

    # Save confusion matrix
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
    plt.title(f'{fault} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f"metrics/{fault}_confusion_matrix.png")
    plt.close()

    print(f"✅ Done: {fault} — Accuracy: {acc:.4f}, Loss: {loss:.4f}")

    # === ROC CURVE ===
    os.makedirs("roc", exist_ok=True)
    fpr, tpr, _ = roc_curve(true_vals, preds)
    roc_auc = roc_auc_score(true_vals, preds)
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{fault} ROC Curve')
    plt.legend(loc='lower right')
    plt.savefig(f"roc/{fault}_roc.png")
    plt.close()

# Save all summary results to CSV
pd.DataFrame(results).T.to_excel("metrics/summary_results.xlsx")



🔍 Training for Fault: A
Found 95 validated image filenames.
Found 10 validated image filenames.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.4812 - loss: 0.8008 - val_accuracy: 0.6526 - val_loss: 0.6177
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.5921 - loss: 0.6969 - val_accuracy: 0.6526 - val_loss: 0.6010
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.7254 - loss: 0.5017 - val_accuracy: 0.8526 - val_loss: 0.5177
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7479 - loss: 0.5534 - val_accuracy: 0.8421 - val_loss: 0.4663
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.8018 - loss: 0.4661 - val_accuracy: 0.7474 - val_loss: 0.4479
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.8258 - loss: 0.3626 - val_accuracy: 0.8316 - val_loss: 0.4139
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.8879 - loss: 0.4220 - val_accuracy: 0.8947 - val_loss: 0.4108
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.8880 - loss: 0.4168 - val_accuracy: 0.9158 - val_loss: 0.3629
Epoch 9/

6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
✅ Done: A — Accuracy: 0.9263, Loss: 0.3341

🔍 Training for Fault: B
Found 95 validated image filenames.
Found 10 validated image filenames.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.5204 - loss: 0.7027 - val_accuracy: 0.8632 - val_loss: 0.4564
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.8871 - loss: 0.4144 - val_accuracy: 0.8632 - val_loss: 0.4659
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.8757 - loss: 0.3802 - val_accuracy: 0.8632 - val_loss: 0.3215
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.8865 - loss: 0.2929 - val_accuracy: 0.8632 - val_loss: 0.3203
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.8810 - loss: 0.2874 - val_accuracy: 0.8632 - val_loss: 0.2761
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8026 - loss: 0.3291 - val_accuracy: 0.8632 - val_loss: 0.2575
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8538 - loss: 0.2698 - val_accuracy: 0.8632 - val_loss: 0.2355
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8033 - loss: 0.2804 - val_accuracy: 0.9158 - val_loss: 0.2172
Epoch 9/

6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step
✅ Done: D — Accuracy: 0.9158, Loss: 0.1865

🔍 Training for Fault: E
Found 95 validated image filenames.
Found 10 validated image filenames.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.7565 - loss: 0.4672 - val_accuracy: 0.8947 - val_loss: 0.4230
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9126 - loss: 0.3470 - val_accuracy: 0.8947 - val_loss: 0.3388
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8976 - loss: 0.3078 - val_accuracy: 0.8947 - val_loss: 0.3012
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9269 - loss: 0.2617 - val_accuracy: 0.8947 - val_loss: 0.2550
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8686 - loss: 0.2855 - val_accuracy: 0.8947 - val_loss: 0.2339
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.8888 - loss: 0.2545 - val_accuracy: 0.8947 - val_loss: 0.2156
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.8629 - loss: 0.2675 - val_accuracy: 0.9053 - val_loss: 0.1986
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.9299 - loss: 0.1660 - val_accuracy: 0.8947 - val_loss: 0.1872
Epoch 9/

6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
✅ Done: E — Accuracy: 0.9053, Loss: 0.1570


In [49]:
# Print metrics nicely
print("🔍 Model Evaluation Summary:\n")
for fault, metrics in results.items():
    print(f"📌 Fault: {fault}")
    print(f"   - Loss: {metrics['loss']:.4f}")
    print(f"   - Accuracy: {metrics['accuracy']:.4f}")
    print("   - Classification Report:")

    # Convert classification report dict to DataFrame for display
    report_df = pd.DataFrame(metrics['classification_report']).transpose()
    display(report_df.round(3)) 
    print("\n" + "-"*60 + "\n")

🔍 Model Evaluation Summary:

📌 Fault: A
   - Loss: 0.3535
   - Accuracy: 0.9263
   - Classification Report:


,precision,recall,f1-score,support
0,0.882,0.909,0.896,33.000
1,0.951,0.935,0.943,62.000
accuracy,0.926,0.926,0.926,0.926
macro avg,0.917,0.922,0.919,95.000
weighted avg,0.927,0.926,0.927,95.000



------------------------------------------------------------

📌 Fault: B
   - Loss: 0.2295
   - Accuracy: 0.9579
   - Classification Report:


,precision,recall,f1-score,support
0,0.951,1.000,0.975,77.000
1,1.000,0.778,0.875,18.000
accuracy,0.958,0.958,0.958,0.958
macro avg,0.975,0.889,0.925,95.000
weighted avg,0.960,0.958,0.956,95.000



------------------------------------------------------------

📌 Fault: C
   - Loss: 0.3860
   - Accuracy: 0.9684
   - Classification Report:


...


------------------------------------------------------------

📌 Fault: D
   - Loss: 0.1408
   - Accuracy: 0.9368
   - Classification Report:


,precision,recall,f1-score,support
0,1.000,0.538,0.700,13.000
1,0.932,1.000,0.965,82.000
accuracy,0.937,0.937,0.937,0.937
macro avg,0.966,0.769,0.832,95.000
weighted avg,0.941,0.937,0.928,95.000



------------------------------------------------------------

📌 Fault: E
   - Loss: 0.1791
   - Accuracy: 0.9368
   - Classification Report:


,precision,recall,f1-score,support
0,0.934,1.000,0.966,85.000
1,1.000,0.400,0.571,10.000
accuracy,0.937,0.937,0.937,0.937
macro avg,0.967,0.700,0.769,95.000
weighted avg,0.941,0.937,0.924,95.000



------------------------------------------------------------



In [52]:
import pandas as pd

# Build a formatted list of rows
formatted_rows = []

for fault, metrics in results.items():
    report = metrics['classification_report']
    row = {
        'Fault': fault,
        'Loss': round(metrics['loss'], 4),
        'Accuracy': round(metrics['accuracy'], 4),
        'Precision_0': round(report['0']['precision'], 4),
        'Recall_0': round(report['0']['recall'], 4),
        'F1_0': round(report['0']['f1-score'], 4),
        'Precision_1': round(report['1']['precision'], 4),
        'Recall_1': round(report['1']['recall'], 4),
        'F1_1': round(report['1']['f1-score'], 4),
        'Macro_F1': round(report['macro avg']['f1-score'], 4),
        'Weighted_F1': round(report['weighted avg']['f1-score'], 4)
    }
    formatted_rows.append(row)

# Convert to DataFrame
formatted_df = pd.DataFrame(formatted_rows)

# Save to CSV
formatted_df.to_excel("metrics/evaluation_summary.xlsx", index=False)

# Display in notebook
display(formatted_df)

,Fault,Loss,Accuracy,Precision_0,Recall_0,F1_0,Precision_1,Recall_1,F1_1,Macro_F1,Weighted_F1
0,A,0.3535,0.9263,0.8824,0.9091,0.8955,0.9508,0.9355,0.9431,0.9193,0.9266
1,B,0.2295,0.9579,0.9506,1.0000,0.9747,1.0000,0.7778,0.8750,0.9248,0.9558
2,C,0.3860,0.9684,0.9545,0.9767,0.9655,0.9804,0.9615,0.9709,0.9682,0.9684
3,D,0.1408,0.9368,1.0000,0.5385,0.7000,0.9318,1.0000,0.9647,0.8324,0.9285
4,E,0.1791,0.9368,0.9341,1.0000,0.9659,1.0000,0.4000,0.5714,0.7687,0.9244


In [38]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tqdm import tqdm

In [41]:
# Fault columns
fault_columns = ['A', 'B', 'C', 'D', 'E']

# Load helper file
df = pd.read_csv("file_names.csv")
df[fault_columns] = df[fault_columns].replace({'T': 1, 'F': 0})  # ensure binary
df['Image'] = df['Image'].apply(lambda x: f"data/images/train/{x}")
# Function to prepare a single image
def prepare_image(image_path):
    try:
        img = load_img(image_path, target_size=(299, 299))
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        return np.expand_dims(img_array, axis=0)
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        return None

In [42]:
df["Image"].head()

0    data/images/train/AH000438.JPG
1    data/images/train/AH015875.JPG
2    data/images/train/AH019263.JPG
3    data/images/train/AH020596.JPG
4    data/images/train/AH026019.JPG
Name: Image, dtype: object

In [43]:
# Make predictions
for fault in fault_columns:
    model_path = f"models/model_{fault}.h5"
    if not os.path.exists(model_path):
        print(f"⚠️ Model not found for {fault}. Skipping.")
        df[fault + '1'] = ""
        continue

    print(f"🔍 Predicting Fault: {fault}")
    model = load_model(model_path)

    predictions = []
    for img_path in tqdm(df['Image']):
        img_tensor = prepare_image(img_path)
        if img_tensor is not None:
            pred = model.predict(img_tensor)[0][0]
            predictions.append(round(pred))
        else:
            predictions.append(np.nan)

    df[fault + '1'] = predictions

# Save new helper with predictions
df.to_csv("final_data.csv", index=False)
print("✅ Predictions appended and saved to helper_with_predictions.csv")


🔍 Predicting Fault: A


  0%|          | 0/105 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


  1%|          | 1/105 [00:06<10:44,  6.20s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


 99%|█████████▉| 104/105 [00:26<00:00,  3.82it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


100%|██████████| 105/105 [00:26<00:00,  3.90it/s]

✅ Predictions appended and saved to helper_with_predictions.csv


In [46]:
df[["Image", "A", "A1", "B", "B1", "C", "C1", "D", "D1", "E", "E1", "Status"]].to_csv("final_data.csv")

import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [63]:
sizes = [25, 60]  # Number of samples to evaluate
os.makedirs("plots")

In [65]:
all_metrics=[]
# === Training & Evaluation Loop ===
for fault in fault_columns:
    print(f"\n🔍 Evaluating Fault: {fault}")

    for size in sizes:
        print(f"\n📊 Training on {size} samples")

        # Data split
        train_df = df.sample(n=size, random_state=42)

        # Generators
        train_gen, val_gen = get_data_generators(train_df, fault, datagen)
        val_gen = train_gen

        # Model
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

        early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        # Training
        history = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=1, callbacks=[early_stop])

        # === Save Plots ===
        plt.figure()
        plt.plot(history.history['accuracy'], label='Train Acc')
        plt.plot(history.history['val_accuracy'], label='Val Acc')
        plt.title(f'{fault} Accuracy ({size} Samples)')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"plots/{fault}_accuracy_{size}.png")
        plt.close()

        plt.figure()
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f'{fault} Loss ({size} Samples)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"plots/{fault}_loss_{size}.png")
        plt.close()

        # === Evaluation ===
        val_gen.reset()
        preds = model.predict(val_gen).ravel()
        preds_binary = (preds > 0.5).astype(int)
        true_vals = val_gen.labels

        report = classification_report(true_vals, preds_binary, output_dict=True)
        loss, acc = model.evaluate(val_gen, verbose=0)

        # === Metric Row ===
        row = {
            "Fault": f"{fault}_{size}",
            "Loss": round(loss, 4),
            "Accuracy": round(acc, 4),
            "Precision_0": round(report['0']['precision'], 4) if '0' in report else None,
            "Recall_0": round(report['0']['recall'], 4) if '0' in report else None,
            "F1_0": round(report['0']['f1-score'], 4) if '0' in report else None,
            "Precision_1": round(report['1']['precision'], 4) if '1' in report else None,
            "Recall_1": round(report['1']['recall'], 4) if '1' in report else None,
            "F1_1": round(report['1']['f1-score'], 4) if '1' in report else None,
            "Macro_F1": round(report['macro avg']['f1-score'], 4),
            "Weighted_F1": round(report['weighted avg']['f1-score'], 4)
        }
        all_metrics.append(row)

# === Save to CSV ===
metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_csv("metrics/evaluation_summary.csv", index=False)
print("\n✅ Evaluation summary saved to metrics/evaluation_summary.csv")



🔍 Evaluating Fault: A

📊 Training on 25 samples
Found 23 validated image filenames.
Found 2 validated image filenames.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 6s/step - accuracy: 0.6729 - loss: 0.6419 - val_accuracy: 0.7391 - val_loss: 0.5464
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.8551 - loss: 0.4921 - val_accuracy: 0.7826 - val_loss: 0.4761
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.7509 - loss: 0.4928 - val_accuracy: 0.8261 - val_loss: 0.4130
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.8007 - loss: 0.4237 - val_accuracy: 0.8261 - val_loss: 0.3732
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.8551 - loss: 0.3330 - val_accuracy: 0.8261 - val_loss: 0.3605
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.8007 - loss: 0.3846 - val_accuracy: 0.8261 - val_loss: 0.3066
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.9130 - loss: 0.2526 - val_accuracy: 0.8696 - val_loss: 0.2757
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.8505 - loss: 0.2950 - val_accuracy: 1.0000 - val_loss: 0.2528
Epoch 9/10
2/2 

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.4681 - loss: 0.7649 - val_accuracy: 0.6111 - val_loss: 0.6535
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.6215 - loss: 0.6282 - val_accuracy: 0.7778 - val_loss: 0.5961
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.7323 - loss: 0.5717 - val_accuracy: 0.6667 - val_loss: 0.5126
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.6410 - loss: 0.5425 - val_accuracy: 0.6852 - val_loss: 0.4726
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.7530 - loss: 0.5047 - val_accuracy: 0.9074 - val_loss: 0.4180
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9200 - loss: 0.4052 - val_accuracy: 0.8704 - val_loss: 0.3860
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.8377 - loss: 0.4153 - val_accuracy: 0.9074 - val_loss: 0.3572
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.8618 - loss: 0.3866 - val_accuracy: 0.9074 - val_loss: 0.3332
Epoch 9/1

...

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 6s/step - accuracy: 0.8505 - loss: 0.4632 - val_accuracy: 0.9130 - val_loss: 0.2754
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.9004 - loss: 0.2986 - val_accuracy: 0.9130 - val_loss: 0.3080
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.9420 - loss: 0.2340 - val_accuracy: 0.9130 - val_loss: 0.3452
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.9420 - loss: 0.2390 - val_accuracy: 0.9130 - val_loss: 0.3441
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step


C:\Users\RLS\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\RLS\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\RLS\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le


📊 Training on 60 samples
Found 54 validated image filenames.
Found 6 validated image filenames.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.7536 - loss: 0.5359 - val_accuracy: 0.8519 - val_loss: 0.4679
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8682 - loss: 0.4212 - val_accuracy: 0.8519 - val_loss: 0.4310
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8164 - loss: 0.5271 - val_accuracy: 0.8519 - val_loss: 0.3382
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8734 - loss: 0.3307 - val_accuracy: 0.8704 - val_loss: 0.3350
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8802 - loss: 0.3081 - val_accuracy: 0.8704 - val_loss: 0.2802
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8414 - loss: 0.2954 - val_accuracy: 0.8519 - val_loss: 0.2702
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8674 - loss: 0.2345 - val_accuracy: 0.8704 - val_loss: 0.2482
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9108 - loss: 0.1850 - val_accuracy: 0.8704 - val_loss: 0.2228
Epoch 9/

In [67]:
metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_excel("metrics/new_evaluation_summary.xlsx", index=False)
print("\n✅ Evaluation summary saved to metrics/evaluation_summary.csv")


✅ Evaluation summary saved to metrics/evaluation_summary.csv


In [78]:
from scipy.stats import chisquare
import pandas as pd

# Load your data
result_df = pd.read_csv("final_data.csv")
display(result_df.head())

# Adjusted Chi-Square test
#for obs, exp in [("A", "A1"), ("B", "B1"), ("C", "C1"), ("D", "D1"), ("E", "E1")]:
observed = result_df[obs]
expected = result_df[exp]

# Scale expected to match observed total
expected_scaled = n.array(expected * (observed.sum() / expected.sum())

# Perform Chi-Square test
chi_stat, p_value = chisquare(f_obs=observed, f_exp=expected_scaled)

print(f"Chi-square test for {obs} vs {exp}")
print("Chi-square Statistic:", chi_stat)
print("P-value:", p_value)
print("-" * 40)


,Unnamed: 0,Image,A,A1,B,B1,C,C1,D,D1,E,E1,Status
0,0,data/images/train/AH000438.JPG,1,0,0,0,1,1,1,1,0,0,Abnormal
1,1,data/images/train/AH015875.JPG,0,0,0,0,0,0,1,1,0,0,Abnormal
2,2,data/images/train/AH019263.JPG,1,0,0,0,1,0,1,1,0,0,Abnormal
3,3,data/images/train/AH020596.JPG,1,1,0,0,0,0,1,1,0,0,Abnormal
4,4,data/images/train/AH026019.JPG,0,0,0,0,0,0,0,1,0,0,Normal


Chi-square test for A vs A1
Chi-square Statistic: nan
P-value: nan
----------------------------------------
Chi-square test for B vs B1
Chi-square Statistic: nan
P-value: nan
----------------------------------------
Chi-square test for C vs C1
Chi-square Statistic: nan
P-value: nan
----------------------------------------
Chi-square test for D vs D1
Chi-square Statistic: nan
P-value: nan
----------------------------------------
Chi-square test for E vs E1
Chi-square Statistic: nan
P-value: nan
----------------------------------------


C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:7985: RuntimeWarning: divide by zero encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:7985: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp


In [79]:
y_test_goodness = [
    1,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,0,1,1,0,0,1,0,
    1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,
    1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,1,1,0,
    1,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,0,0,0,0,
    1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,
    1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,
    0,1,0,1,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,
    1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,
    1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,
    0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
]

y_pred_goodness = [
    0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,0,
    1,0,0,0,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,
    1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,
    1,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,0,0,
    0,0,1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,
    0,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,0,
    1,0,1,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,
    1,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
]


In [81]:
import numpy as np
from scipy.stats import chisquare
from sklearn.metrics import confusion_matrix


# Get the observed counts from predictions
obs_counts = np.bincount(y_pred_goodness)  # Count predicted 0s and 1s

# Get the expected counts from actual labels
exp_counts = np.bincount(y_test_goodness)  # Count actual 0s and 1s

# Ensure both arrays have the same length (in case one class is missing)
max_len = max(len(obs_counts), len(exp_counts))
obs_counts = np.pad(obs_counts, (0, max_len - len(obs_counts)))
exp_counts = np.pad(exp_counts, (0, max_len - len(exp_counts)))

# Scale expected counts so totals match observed counts
exp_counts_scaled = exp_counts * (obs_counts.sum() / exp_counts.sum())

# Run Chi-square test
chi_stat, p_value = chisquare(f_obs=obs_counts, f_exp=exp_counts_scaled)

print("Observed counts:", obs_counts)
print("Expected counts (scaled):", exp_counts_scaled.round(2))
print("Chi-square statistic:", chi_stat)
print("p-value:", p_value)

# Interpretation
if p_value < 0.05:
    print("Reject H0: Predicted distribution is significantly different from actual distribution.")
else:
    print("Fail to reject H0: Predicted distribution is similar to actual distribution.")

Observed counts: [312 184]
Expected counts (scaled): [285.94 210.06]
Chi-square statistic: 5.607928135748498
p-value: 0.017879391027055807
Reject H0: Predicted distribution is significantly different from actual distribution.
